In [12]:
import torch
import torchvision.models as models
from torchvision import transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split
import torch.optim as optim
import torch.nn.functional as F
from torch import nn
import pandas as pd
import cv2
import imghdr
from sklearn.metrics import classification_report
from matplotlib import pyplot as plt
from tqdm import tqdm

import os
for dirname, _, filenames in os.walk('/kaggle/input/animal-image-dataset-90-different-animals/animals/animals'):
    for filename in filenames:
        file_path = os.path.join(dirname, filename)
        try:
            img = cv2.imread(file_path)
            tip = imghdr.what(file_path)
#             print(tip)
            if tip != 'jpeg':
                print('Image not in ext list {}'.format(file_path))
                os.remove(file_path)
        except Exception as e:
            print('Issue with image {}'.format(file_path))
        

In [13]:
# CUDA_LAUNCH_BLOCKING=1
# TORCH_USE_CUDA_DSA

In [14]:
data_dir = '/kaggle/input/animal-image-dataset-90-different-animals/animals/animals'

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(3),  # Convert to RGB (3 channels)
    transforms.ToTensor(),
#     transforms.Normalize(mean=[0.5], std=[0.5]),
])

dataset = datasets.ImageFolder(root = data_dir, transform=transform)
class_names = dataset.classes
print(class_names)

['antelope', 'badger', 'bat', 'bear', 'bee', 'beetle', 'bison', 'boar', 'butterfly', 'cat', 'caterpillar', 'chimpanzee', 'cockroach', 'cow', 'coyote', 'crab', 'crow', 'deer', 'dog', 'dolphin', 'donkey', 'dragonfly', 'duck', 'eagle', 'elephant', 'flamingo', 'fly', 'fox', 'goat', 'goldfish', 'goose', 'gorilla', 'grasshopper', 'hamster', 'hare', 'hedgehog', 'hippopotamus', 'hornbill', 'horse', 'hummingbird', 'hyena', 'jellyfish', 'kangaroo', 'koala', 'ladybugs', 'leopard', 'lion', 'lizard', 'lobster', 'mosquito', 'moth', 'mouse', 'octopus', 'okapi', 'orangutan', 'otter', 'owl', 'ox', 'oyster', 'panda', 'parrot', 'pelecaniformes', 'penguin', 'pig', 'pigeon', 'porcupine', 'possum', 'raccoon', 'rat', 'reindeer', 'rhinoceros', 'sandpiper', 'seahorse', 'seal', 'shark', 'sheep', 'snake', 'sparrow', 'squid', 'squirrel', 'starfish', 'swan', 'tiger', 'turkey', 'turtle', 'whale', 'wolf', 'wombat', 'woodpecker', 'zebra']


In [15]:
loader = DataLoader(dataset, batch_size=64, shuffle=True)

mean_sum = torch.zeros(3)
std_sum = torch.zeros(3)
count = 0

# Calculate mean and standard deviation for each channel
for images, _ in loader:
    batch_size = images.size(0)
    count += batch_size
    mean_sum += images.mean(dim=[0, 2, 3]) * batch_size
    std_sum += images.std(dim=[0, 2, 3]) * batch_size

# Calculate the overall mean and standard deviation
mean = mean_sum / count
std = std_sum / count

print(f"Estimated Mean: {mean}")
print(f"Estimated Std: {std}")
    




Estimated Mean: tensor([0.4962, 0.4962, 0.4962])
Estimated Std: tensor([0.2631, 0.2631, 0.2631])


In [16]:
normalize_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(3),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

dataset = datasets.ImageFolder(root=data_dir, transform=normalize_transform)
# for image, labels in dataloader:
#     print(labels)

In [17]:
train_size = int(len(dataset)*.7)
# valid_size = int(len(dataset)*.2)
test_size = int(len(dataset) - train_size)

train, test = random_split(dataset, [train_size, test_size])

batch_size = 16
train_dataloader = DataLoader(train, batch_size=batch_size, shuffle=True)
# valid_dataloader = DataLoader(valid, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test, batch_size=batch_size, shuffle=False)
# transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.Grayscale(3),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.5], std=[0.5]),
# ])

# mnist_train_dataset = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
# mnist_test_dataset = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)

# train_loader = DataLoader(mnist_train_dataset, batch_size=32, shuffle=True)
# test_loader = DataLoader(mnist_test_dataset, batch_size=32, shuffle=False)

In [18]:
class CNNModel(nn.Module):
    def __init__(self, num_classes = 90):
        super(CNNModel, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 64, kernel_size = 3, stride = 1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size = 3, stride =1 )
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, kernel_size = 3, stride = 1)
        self.bn3 = nn.BatchNorm2d(256)
        
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.fc1 = nn.Linear(256*26*26, 512)
        self.bn4 = nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(512, num_classes)
        self.bn5 = nn.BatchNorm1d(num_classes)
        
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
#         print(x.size())
        x = self.pool(self.relu(self.bn1(self.conv1(x))))
#         print(x.size())
        x = self.pool(self.relu(self.bn2(self.conv2(x))))
#         print(x.size())
        x = self.pool(self.relu(self.bn3(self.conv3(x))))
#         print(x.size())
        x = x.view(-1, 256*26*26)
#         print(x.size())
        x = self.relu(self.bn4(self.fc1(x)))
#         print(x.size())
        x = self.bn5(self.fc2(x))
#         x = F.log_softmax(x, 1)
#         print(x.size())
        
        return x

model = CNNModel().to('cuda')
criterion = nn.CrossEntropyLoss()
optimiser = optim.Adam(model.parameters(), lr=1e-5, weight_decay=1e-5)

# alexnet_model = models.alexnet(pretrained=True)

# num_features = alexnet_model.classifier[6].in_features
# alexnet_model.classifier[6] = nn.Linear(num_features, 90)
# alexnet_model = alexnet_model.to("cuda")
# alexnet_model.train()

# num_ftrs = alexnet_model.fc.in_features
# alexnet_model.fc = torch.nn.Linear(num_ftrs, 90)

# alexnet_model = models.vgg16(pretrained=True)

# # Modify the classifier to match your number of classes
# num_features = alexnet_model.classifier[6].in_features
# alexnet_model.classifier[6] = nn.Linear(num_features, 90)
# alexnet_model = alexnet_model.to("cuda")
# alexnet_model.train()

In [19]:
# for param in alexnet_model.parameters():
#     param.requires_grad = False
    
# for param in alexnet_model.classifier.parameters():
#     param.requires_grad = True

In [20]:
# criterion = nn.CrossEntropyLoss()
# optimiser = optim.Adam(alexnet_model.parameters(), lr=1e-5, weight_decay=1e-5)

In [21]:
num_epoch = 10
# for epoch in range(num_epoch):
#     alexnet_model.train()
#     train_loss = 0.0
#     for images, labels in train_dataloader:
#         images, labels = images.to('cuda'), labels.to('cuda')
#         optimiser.zero_grad()
#         output = alexnet_model(images)
# #         output = output.float()
# #         labels = labels.long()
# #         print(output)
#         loss = criterion(output, labels)
#         loss.backward()
#         optimiser.step()
#         total_loss = loss.item()
        
#     avg_loss = total_loss/len(train_dataloader)
#     print(f'Training Epoch [{epoch+1}/{num_epoch}], Loss: {avg_loss}')
    


for epoch in range(num_epoch):
    for inputs, labels in train_dataloader:
        optimiser.zero_grad()
        inputs, labels = inputs.to("cuda"), labels.to("cuda")
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimiser.step()
        
    print(f"Epoch [{epoch+1}/{num_epoch}], Loss: {loss.item()}")
    
# torch.save(model.state_dict(), 'custom_cnn_model_gpu.pth')

Epoch [1/10], Loss: 3.556795358657837
Epoch [2/10], Loss: 3.6404407024383545
Epoch [3/10], Loss: 3.5529167652130127
Epoch [4/10], Loss: 3.5296106338500977
Epoch [5/10], Loss: 3.977418899536133
Epoch [6/10], Loss: 3.4618682861328125
Epoch [7/10], Loss: 3.522200584411621
Epoch [8/10], Loss: 3.3904449939727783
Epoch [9/10], Loss: 3.4630165100097656
Epoch [10/10], Loss: 3.5220754146575928


Epoch [1/10], Loss: 0.6496276259422302
Epoch [2/10], Loss: 0.025630353018641472
Epoch [3/10], Loss: 0.035333409905433655
Epoch [4/10], Loss: 0.06272778660058975
Epoch [5/10], Loss: 0.014833853580057621
Epoch [6/10], Loss: 0.0015034597599878907
Epoch [7/10], Loss: 0.0040255882777273655
Epoch [8/10], Loss: 0.00011664794146781787
Epoch [9/10], Loss: 0.0008121779537759721
Epoch [10/10], Loss: 0.03196653351187706


Epoch [1/10], Loss: 8.662488653499167e-06
Epoch [2/10], Loss: 6.5564759097469505e-06
Epoch [3/10], Loss: 0.00017892976757138968
Epoch [4/10], Loss: 1.8635873857419938e-05
Epoch [5/10], Loss: 1.839747892518062e-05
Epoch [6/10], Loss: 7.362770702457055e-05
Epoch [7/10], Loss: 0.00014585816825274378
Epoch [8/10], Loss: 0.007957431487739086
Epoch [9/10], Loss: 0.0009501574095338583
Epoch [10/10], Loss: 0.0009352660272270441

For AlexNet
Epoch [1/5], Loss: 0.0
Epoch [2/5], Loss: 0.0
Epoch [3/5], Loss: 0.0
Epoch [4/5], Loss: 0.0
Epoch [5/5], Loss: 0.0

For ResNet
Epoch [1/5], Loss: 2.1291191577911377
Epoch [2/5], Loss: 2.1237082481384277
Epoch [3/5], Loss: 2.1333234310150146
Epoch [4/5], Loss: 2.130035161972046
Epoch [5/5], Loss: 2.127274751663208

In [24]:
# alexnet_model.eval()
for epoch in range(num_epoch):
    
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in tqdm(test_dataloader):
    #         print(images)
    #         fig, axes = plt.subplots(1,1)
    #         axes.imshow(images)
    #         plt.show()
    #         plt.imshow(images)
    #         t_i = normalize_transform(images)
    #         actual_batch_size = images.size(0)
    #         plt.figure(figsize=(15, 15))  # Adjust figure size as needed
    #         for i in range(actual_batch_size):
    #             plt.subplot(4, 8, i + 1)  # Assuming 32 images will be displayed in a 4x8 grid
    # #             plt.imshow(transforms.ToPILImage()(images[i]))
    #             plt.imshow((images[i].permute(1, 2, 0)))# Convert tensor back to PIL Image for plotting
    #             plt.title(f'Label: {class_names[labels[i]]}')
    #             plt.axis('off')
    #         plt.show()
            images, labels = images.to('cuda'), labels.to('cuda')

            optimiser.zero_grad()
            output = model(images)
    #         i, predicted = torch.max(output.data, 1)
            predicted = torch.argmax(torch.softmax(output, dim=1), dim=1)

    #         print(labels)
    #         for i in range(actual_batch_size):
    #             print(class_names[predicted[i]])
    #         print(predicted)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

        accuracy = correct/total
        print(f"Epoch {epoch + 1}/{num_epoch}, Validation Accuracy: {accuracy * 100:.2f}%")

100%|██████████| 102/102 [00:23<00:00,  4.37it/s]


Epoch 1/10, Validation Accuracy: 30.35%


100%|██████████| 102/102 [00:23<00:00,  4.37it/s]


Epoch 2/10, Validation Accuracy: 30.35%


100%|██████████| 102/102 [00:23<00:00,  4.32it/s]


...

Epoch 8/10, Validation Accuracy: 30.35%


100%|██████████| 102/102 [00:23<00:00,  4.40it/s]


Epoch 9/10, Validation Accuracy: 30.35%


100%|██████████| 102/102 [00:23<00:00,  4.40it/s]

Epoch 10/10, Validation Accuracy: 30.35%


Epoch 1/10, Validation Accuracy: 78.78%
Epoch 2/10, Validation Accuracy: 78.78%
Epoch 3/10, Validation Accuracy: 78.78%
Epoch 4/10, Validation Accuracy: 78.78%
Epoch 5/10, Validation Accuracy: 78.78%
Epoch 6/10, Validation Accuracy: 78.78%
Epoch 7/10, Validation Accuracy: 78.78%
Epoch 8/10, Validation Accuracy: 78.78%
Epoch 9/10, Validation Accuracy: 78.78%
Epoch 10/10, Validation Accuracy: 78.78%




Epoch 1/10, Validation Accuracy: 80.38%
Epoch 2/10, Validation Accuracy: 80.38%
Epoch 3/10, Validation Accuracy: 80.38%
Epoch 4/10, Validation Accuracy: 80.38%
Epoch 5/10, Validation Accuracy: 80.38%
Epoch 6/10, Validation Accuracy: 80.38%
Epoch 7/10, Validation Accuracy: 80.38%
Epoch 8/10, Validation Accuracy: 80.38%
Epoch 9/10, Validation Accuracy: 80.38%
Epoch 10/10, Validation Accuracy: 80.38%

For AlexNet
Epoch 1/5, Validation Accuracy: 100.00%
Epoch 2/5, Validation Accuracy: 100.00%
Epoch 3/5, Validation Accuracy: 100.00%
Epoch 4/5, Validation Accuracy: 100.00%
Epoch 5/5, Validation Accuracy: 100.00%

For ResNet
Epoch 1/5, Validation Accuracy: 0.00%
Epoch 2/5, Validation Accuracy: 0.00%
Epoch 3/5, Validation Accuracy: 0.00%
Epoch 4/5, Validation Accuracy: 0.00%
Epoch 5/5, Validation Accuracy: 0.00%

In [ ]:
# alexnet_model.eval()
# for epoch in range(num_epoch):
    
#     correct = 0
#     total = 0
#     for images, labels in spiral_dataloader:
# #         print(images)
# #         fig, axes = plt.subplots(1,1)
# #         axes.imshow(images)
# #         plt.show()
# #         plt.imshow(images)
# #         t_i = normalize_transform(images)
#         actual_batch_size = images.size(0)
#         plt.figure(figsize=(15, 15))  # Adjust figure size as needed
#         for i in range(actual_batch_size):
#             plt.subplot(4, 8, i + 1)  # Assuming 32 images will be displayed in a 4x8 grid
# #             plt.imshow(transforms.ToPILImage()(images[i]))
#             plt.imshow((images[i].permute(1, 2, 0)))# Convert tensor back to PIL Image for plotting
#             plt.title(f'Label: {class_names[labels[i]]}')
#             plt.axis('off')
#         plt.show()
#         images, labels = images.to('cuda'), labels.to('cuda')
        
#         optimiser.zero_grad()
#         output = alexnet_model(images)
#         i, predicted = torch.max(output.data, 1)
        
# #         print(labels)
#         for i in range(actual_batch_size):
#             print(class_names[predicted[i]])
# #         print(predicted)
#         correct += predicted.eq(labels).sum().item()
#         total += labels.size(0)
    
#     accuracy = correct/total
#     print(f"Epoch {epoch + 1}/{num_epoch}, Validation Accuracy: {accuracy * 100:.2f}%")

In [ ]:
# alexnet_model.eval()
# y_true=[]
# y_pred=[]
# with torch.no_grad():
#     for test_data in test_dataloader:
#         images, labels = images.to('cuda'), labels.to('cuda')
#         pred = alexnet_model(images).argmax(dim=1)
#         for i in range(len(pred)):
#             y_true.append(labels[i].item())
#             y_pred.append(pred[i].item())
# print(len(y_true))
# print(classification_report(y_true,y_pred,target_names=class_names,digits=4))